# 题目：顺时针打印矩阵
输入一个矩阵，按照从外向里，以顺时针的顺序依次打印出每一个数字。例如矩阵：  
1  2  3  4  
5  6  7  8  
9  10 11 12  
13 14 15 16    
则依次打印出数字[1,2,3,4,8,12,16,15,14,13,9,5,6,7,11,10]

## 分析：
题目并没有涉及什么复杂的数据结构或者高级的算法，但实际上解决这个问题会在代码中包含多个循环，并且需要判断多个边界条件。因此解决这个问题的关键在于先形成清洗的思路，并把复杂的问题分解成若干个简单的问题。  
对于本问题而言，实际上可以将矩阵看成若干个圆圈，每次都打印最外层的圈。那么接下来就要分析循环结束的条件：  

假设这个矩阵的行数是rows，列数是cols。打印第一圈的左上角左边一定是$(0,0)$，第二圈的左上角坐标是$(1,1)$，依次类推。注意到左上角的横纵坐标都是一样的，于是可以在矩阵中选取左上角$(start,start)$的一圈作为我们分析的目标。  
对于一个$5*5$的矩阵而言，最后一圈只有一个数字，对应的坐标为$(2,2)$，我们发现$5>2*2$；对于一个$6*6$的矩阵而言，最后一圈有四个数字，其左上角的坐标仍然为$(2,2)$，并且我们发现$6>2*2$仍然成立。于是得出，让循环继续的条件是$cols>startX*2$并且$rows>startY*2$。这样我们就可以写一个循环来打印矩阵了。  

接下来我们又要考虑如何打印一圈的功能。我们可以把打印一圈分成四步：第一步，从左到右打印一行；第二步，从上到下打印一列；第三步，从右到左打印一行；第四步，从下到上打印一列。每一步我们都可以根据起始坐标和终止坐标用一个循环就能打印一行或者一列。  
但是需要注意的是，最后一圈有可能退化成只有一行（列），甚至只有一个数字，这时打印一圈就不需要四步了。（考虑长宽不同的时候）。  
因此，我们要仔细分析打印时每一步的前提条件。第一步总是需要的。如果只有一行，那么就不需要第二步了：也就是说，需要第二步的前提条件是终止行号大于起始行号。需要第三步的前提条件是圈内至少有两行两列。第四步的前提条件是至少有三行两列，因此要求终止行号比起始行号至少大2，同时终止列号大于起始列号。

In [34]:
class Solution:
    def PrintMatrix(self,data,cols,rows):
        if data is None or cols<=0 or rows<=0:
            return 
        start = 0
        while(cols>start*2 and rows>start*2):
            self.PrintMatrixInCircle(data,cols,rows,start)
            start+=1
            
    def PrintMatrixInCircle(self,data,cols,rows,start):
        endX = cols-1-start#最后一列
        endY = rows-1-start#最后一行
        
        #第一步：从左到右打印一行
        for i in range(start,endX+1):
            num = data[start][i]
            print(num,end=' ')
            
        #第二步：从上到下打印一列
        if start<endY:
            for i in range(start+1,endY+1):
                num = data[i][endX]
                print(num,end=' ')
        
        #第三步：从右到左打印一行
        if start<endY and start<endX:
            for i in range(endX-1,start-1,-1):
                num = data[endY][i]
                print(num,end=' ')
        
        #第四步：从下到上打印一列
        if start<endX and start<endY-1:
            for i in range(endY-1,start,-1):
                num = data[i][start]
                print(num,end=' ')

# 测试

In [17]:
import numpy as np

In [23]:
data = np.array(list(range(1,17)))
data = data.reshape(4,4).tolist()
data

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]

In [24]:
Solution().PrintMatrix(data,4,4)

1 2 3 4 8 12 16 15 14 13 9 5 6 7 11 10 

In [25]:
data1 = np.array(list(range(1,26)))
data1 = data1.reshape(5,5).tolist()
data1

[[1, 2, 3, 4, 5],
 [6, 7, 8, 9, 10],
 [11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20],
 [21, 22, 23, 24, 25]]

In [26]:
Solution().PrintMatrix(data1,5,5)

1 2 3 4 5 10 15 20 25 24 23 22 21 16 11 6 7 8 9 14 19 18 17 12 13 

In [28]:
data2 = np.array(list(range(1,7)))
data2 = data2.reshape(2,3).tolist()
data2

[[1, 2, 3], [4, 5, 6]]

In [35]:
Solution().PrintMatrix(data2,3,2)

1 2 3 6 5 4 

## 解法二： Python的黑魔法
https://www.cnblogs.com/yanmk/p/8980173.html

思路很独特：每次取矩阵的第一行，取完之后矩阵去掉第一行得到下一个矩阵。新的矩阵逆时针旋转，再取新矩阵的第一行，同样去掉第一行得到下一个矩阵。重复操作直到矩阵变为空。

In [36]:
class Solution:
    def spiralOrder(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: List[int]
        """
        #1.短路机制：当matrix==[]时就不再执行and之后的运算了
        #2.matrix.pop(0)取第一行
        #3.* 星号表达式，python特有
        return matrix and [*matrix.pop(0)] + self.spiralOrder([*zip(*matrix)][::-1])

In [37]:
data2 = np.array(list(range(1,7)))
data2 = data2.reshape(2,3).tolist()
data2

[[1, 2, 3], [4, 5, 6]]

In [38]:
Solution().spiralOrder(data2)

[1, 2, 3, 6, 5, 4]